<a href="https://colab.research.google.com/github/KSG722/class2022Spring/blob/main/audio_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio Processing
---

https://musiclab.chromeexperiments.com/Spectrogram/

### Fourier transform (분석용)
* spectrum은 주어진 signal에 대해 어떤 주파수 성분이 많이 있나? 
->x축은 freq이므로 x축을 보고 높은 주파수가 많은지, 낮은 주파수가 많은지 구분
* spectrogram은 spectrum을 time 축으로 concatenate한 것
-> 즉 spectrum은 특정 time의 (amplitude와)freqency를 나타낸 것(spectrum은 x축이 frequency, y축이 amplitude). 근데 이걸 시간대별(시간 축)로 쪼르르륵 한 장. 그리고 이 한 장 한 장 이어 붙임. 이어붙인 걸  spectrogram이라 부르는데, x축은 time, y축은 frequency. 
-> spectrogrm : concatenation of spectrums over time

* Fourie Transform 방법: signal (inner product) a series of complex phasors with different frequencies
-> 다양한 주파수에 해당하는 phasor, 뭐 한 500개(10hz, 20hz.... 5000hz)를 우리가 원하는 signal(given signal, 주어진 시그널)에다가 각각 plotting해서 곱해줌.
-> x축이 frequence가 된다.

* inner product는 일종의 correlation (즉, 해당 frequency가 얼마나 있는지 probing) 
-> 예를 들어 10Hz의 complex phasor와 이 signal(inner product)의 correlation이 0.01이 나오면 해당 스펙토그램엔 10hz가 거의 없다는 것. 즉 inner product = 10hz가 있는지 없는지 questioning하고 답을 얻는 것.

* 왜? sine phasor 안 쓰나? phase sensitivity 때문
-> sine phasor의 약점 때문에 complex phasor 사용

### Filter (변환용)
* A --> function -->  B
* signal A --> filter --> signal B
* 신호 (time function)를 입력으로 하는 함수를 filter라고 함
* 왜? (함수가 아니라) filter 라고 부름? 이 함수의 목적이 특정 주파수에 대한 manipulation(조작)이므로.
(예: 어떤 주파수대를 작게, 크게, 통과, 제거 등 -> 저음을 작게 만들고 싶을 때, 낮은 주파수를 작게 or 제거 설정할 수 있다는 것)

* 방법: weighted sum of signal's shifts (두가지 방법: FIR, IIR)

  * FIR: Y(k) = b<sub>1</sub>X(k) + b<sub>2</sub>X(k-1) + ...  -> b<sub>1</sub>, b<sub>2</sub>의 값만 알고 있으면 됨. 
  
원래 시그널 = X(k) , 
시그널을 x축 기준 한 칸 민 것 = X(k-1)

b계수를 1/2로 설정하면
훨씬 본래 그래프보다 완만해짐(audio_processing강의 50분 대 참고). 이런 식으로 필터를 거쳐서 b1,b2에 맞게
시그널이 변형되는 것.

    - Y = H * X

->Y는 X를 H라는 필터에 곱한 것


  * IIR: a<sub>1</sub>Y(k) + a<sub>2</sub>Y(k-1) + ... = X(k) -> FIR과 반대로 X가 아니라 Y에 a<sub>1</sub>, a<sub>2</sub>가 곱해진 모습. a<sub>1</sub>, a<sub>2</sub>를 알고 있으면 됨.

    - Y = (1/H) * X -> 필터가 FIR과 역함수 관계

참고) a<sub>1</sub> = 1, a<sub>2</sub> 이하 = 0일 경우 filter의 효과가 없이 원 데이터가 나온다.

-> Fourier transform은 분석용, Filter는 조작 및 변환용이라는 것에 유의

### Auto correlation
* measuring pitch / F0

->  원 데이터와 해당 데이터를 time lag먹은 데이터 간의 corrleation. 일반적으로 음성 데이터들은 반복되기 때문에 예컨데 원 데이터와 원 데이터를 n초 늦춘 데이터는 나름 비슷하기 때문에 상관계수가 꽤 높게 나온다.

x축은 time, y축은 correlation이라고 한다면
x=0인 시점에서 y =1, 그리고 x=n이 될때까지 y는 감소하다가 x=n인 시점에서 y가 높게 올라감(1까지는 아니겠지만). 이후 2n, 3n으로 쭉 반복.(휴대폰 스샷 참고)

뭐 여기서 1/first lag peak 를 하면 pitch가 구해짐



### RMS: root mean square
* measuring intensity

-> mean을 제곱해서 그 위에 루트 씌우기 -> 진폭이 클 수록 높은 값. 즉 RMS는 진폭을 나타냄. 


->Auto correlation은 음의 pitch, RMS은 intensity과 관련.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa, librosa.display

librosa : 음성파일과 관련된 라이브러리

### load / plot / play sound file

In [ ]:
# from google.colab import files
# fn = files.upload()

컴퓨터에 있는 음성 파일 땡겨오는 함수들

In [ ]:
# from google.colab import files
# fn = files.upload()
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/aeiou.wav"
os.system("curl " + url + " > aeiou.wav")

s, sr = librosa.load('aeiou.wav')

그냥 그냥 github상에 있는 음성 파일 땡겨오는 함수들

s와 sr 위에 커서를 가져다 되면 narray 등에 대한 정보를 볼 수 있음. (48600, 22050)

In [ ]:
s = librosa.util.normalize(s)
librosa.display.waveplot(s, sr)
ipd.Audio(s[7000:12000], rate=sr)

normalize : 시그널을 y축 기준으로 -1와 1 사이에 scale up or scale down 해줌

원래 시그널을 보면 0.4와 -0.4 사이로 시그널이 위치. 이를 이 때 모든 값을 0.4(절대값의 최대값)로 나눔으로써 -1와 1 사이에 위치시킴

s[7000:12000] : '아이에오우' 중에서 frequency가 '아' 정도에 해당하는 7000hz~12000hz

저기서 ipd.Audio(s, rate=sr) 라고 입력하면 아에이오우가 들림

### Fourier Transform for Spectrogram

In [ ]:
s_preemp = librosa.effects.preemphasis(s)

n_fft=512
hop_length=int(0.001*sr)
win_length=int(sr*0.008)

spec = librosa.stft(s_preemp, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window = 'hann')
magspec = np.abs(spec)
dBspec = librosa.amplitude_to_db(magspec, ref=np.max)

plt.figure(figsize=(15, 5))
librosa.display.specshow(dBspec, sr=sr, x_coords = np.linspace(1, len(s), dBspec.shape[1])/sr , x_axis='time', y_axis='linear', cmap='Greys')
plt.ylim((0,5000))

스펙토그램을 만드는 방법. 스펙트럼을 여러 개 만들어야지 그 걸 합쳐서 스펙토그램화하지.
위의 그래프에서 x축을 기준으로 엄청 잘게 쪼갠 거 하나가 스펙트럼. 

그 한 장의 길이가 win_length의 0.008초.

0.08초 짜리 스펙트럼이 겹쳐있음. 0.01초의 hop_length로 차이를 냄. 즉 0.07초가 겹쳐있는 것.

fft =512는 x축을 512등분한다는 의미.

이까지하면 파라미터 선정 끝.

stft는 ft에서 푸리에 transform을 의미.
참고로 complex phasor를 쓰기 때문에 a + bi 형태이고 실수부 a, 허수부 b를 해서 (a,b)

(a,b)와 원점 사이의 거리를 abs로 절대값화. 즉 원점으로부터 거리. 복소수의 원점으로부터의 거리. abs를 넣었기에 복소수에서 실수가 되는 것.. 이러면 스펙트럼 완성.

푸리에 트랜스폼의 용도은 분석용. 어떤 소리 안에 어떤 주파수가 많이 들어있는가를 분석해보는 것. 높은 주파수가 많이 있는지, 낮은 주파수가 많이 있는지 찔러볼 수 있다. 그리고 찔러볼 때는 complex phasor를 씀.

### Filter for audio transformation

In [ ]:
from scipy.signal import lfilter
sig = s[7000:12000]
sig = lfilter(np.array([1]), np.array([1]), sig, axis=0)
librosa.display.waveplot(sig, sr)
ipd.Audio(sig, rate=sr)

lfilter <- linear filter

b 넣고 , a 넣고, 소리넣으면 됨.

b와 a는 계수들의 array.

참고로 여기서는 계수를 b1,a1을 1로 설정하고 나머지 계수들은 0으로 설정했기에(디폴트값) 소리가 변형되지 않는다.

1/2로 설정하면 소리가 부드러워짐. 그래프가 완만해지기 때문.

### RMS(Root Mean Square) for intensity

In [ ]:
rms = librosa.feature.rms(s)
plt.plot(rms[0])

### Autocorrelation for pitch(F0) measurement

In [ ]:
F0, voiced_flag, voiced_prob = librosa.pyin(s, 60, 200)
plt.plot(F0, '.')